# Setup

In [1]:
# import your standard packages
%run ../pkgs.py

# import your local functions
sys.path.insert(1, '../')
from local_functions import *

# make sure the figures plot inline rather than at the end
%matplotlib inline

Default libraries loaded.


# Paths and Parameters

In [2]:
outfn = '../../data/woa_to_wod.nc'

# Read Data

In [3]:
ds_WOA = xr.open_dataset('../../data/woa_processed.nc')
ds_WOA

<xarray.Dataset>
Dimensions:                    (lat: 35, lon: 55, month: 12, pres: 57)
Coordinates:
  * month                      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * pres                       (pres) float32 0.0 5.0 10.0 ... 1450.0 1500.0
  * lon                        (lon) float32 48.5 49.5 50.5 ... 101.5 102.5
  * lat                        (lat) float32 -1.5 -0.5 0.5 ... 30.5 31.5 32.5
Data variables:
    temp                       (month, pres, lat, lon) float64 ...
    sal                        (month, pres, lat, lon) float64 ...
    doxy                       (month, pres, lat, lon) float64 ...
    nitrate                    (month, pres, lat, lon) float64 ...
    phosphate                  (month, pres, lat, lon) float64 ...
    temp_40_200                (month, lat, lon) float64 ...
    doxy_40_200                (month, lat, lon) float64 ...
    sal_40_200                 (month, lat, lon) float64 ...
    nitrate_40_200             (month, lat, lon) float64 ...
 

In [6]:
ds_WOD = xr.open_dataset('../../data/wod_processed.nc')
ds_WOD

<xarray.Dataset>
Dimensions:           (pres: 57, time: 314731)
Coordinates:
  * time              (time) datetime64[ns] 1770-01-01 1770-01-01 ... 2020-05-31
  * pres              (pres) int64 0 5 10 15 20 25 ... 1300 1350 1400 1450 1500
Data variables:
    temp              (time, pres) float64 ...
    sal               (time, pres) float64 ...
    doxy              (time, pres) float64 ...
    nitrate           (time, pres) float64 ...
    phosphate         (time, pres) float64 ...
    lat               (time) float32 ...
    lon               (time) float32 ...
    temp_40_200       (time) float64 ...
    doxy_40_200       (time) float64 ...
    sal_40_200        (time) float64 ...
    nitrate_40_200    (time) float64 ...
    phosphate_40_200  (time) float64 ...
    temp_50_200       (time) float64 ...
    doxy_50_200       (time) float64 ...
    sal_50_200        (time) float64 ...
    nitrate_50_200    (time) float64 ...
    phosphate_50_200  (time) float64 ...
    temp_50_225    

# Interpolate to WOD profiles

In [11]:
xx,yy = np.meshgrid(ds_WOA.lon,ds_WOA.lat)
xx = xx.flatten()
yy = yy.flatten()

lons = np.array(ds_WOD.lon)
lats = np.array(ds_WOD.lat)
months = np.array(ds_WOD.time.dt.month)

TEMP_WOD = np.zeros((len(months),len(ds_WOD.pres)))*np.nan
DOXY_WOD = np.zeros((len(months),len(ds_WOD.pres)))*np.nan
SAL_WOD = np.zeros((len(months),len(ds_WOD.pres)))*np.nan
NITRATE_WOD = np.zeros((len(months),len(ds_WOD.pres)))*np.nan
PHOSPHATE_WOD = np.zeros((len(months),len(ds_WOD.pres)))*np.nan
lon_WOD = np.zeros(len(months))*np.nan
lat_WOD = np.zeros(len(months))*np.nan
month_WOD = np.array(months)
month_WOA = np.arange(1,13)

n = len(month_WOA)
m = len(ds_WOA.pres)

with tqdm(total=n * m) as pbar:
    for tt in range(n):
        for dd in range(m):

            temp = np.array(ds_WOA.temp[tt,dd,:,:])
            sal = np.array(ds_WOA.sal[tt,dd,:,:])
            doxy = np.array(ds_WOA.doxy[tt,dd,:,:])
            nitrate = np.array(ds_WOA.nitrate[tt,dd,:,:])
            phosphate = np.array(ds_WOA.phosphate[tt,dd,:,:])
            
            
            indq = months == np.array(month_WOA[tt])   
            lonq = lons[indq]
            latq = lats[indq]

            # find the lons and lats of TCD vals at this time
            points = np.array( (xx,yy) ).T
            temp_values = temp.flatten()
            sal_values = sal.flatten()
            doxy_values = doxy.flatten()
            nitrate_values = nitrate.flatten()
            phosphate_values = phosphate.flatten()

            TEMP_WOD[indq,dd] = griddata(points, temp_values, (lonq,latq) ,method='linear')
            SAL_WOD[indq,dd] = griddata(points, sal_values, (lonq,latq) ,method='linear')
            DOXY_WOD[indq,dd] = griddata(points, doxy_values, (lonq,latq) ,method='linear')
            NITRATE_WOD[indq,dd] = griddata(points, nitrate_values, (lonq,latq) ,method='linear')
            PHOSPHATE_WOD[indq,dd] = griddata(points, phosphate_values, (lonq,latq) ,method='linear')
            
            lon_WOD[indq] = lonq
            lat_WOD[indq] = latq
            
            pbar.update(1)

# set the places that are nans in WOD temp or doxy to nan
ind = np.isnan(ds_WOD.temp)
TEMP_WOD[ind] = np.nan

ind = np.isnan(ds_WOD.doxy)
DOXY_WOD[ind] = np.nan

ind = np.isnan(ds_WOD.sal)
SAL_WOD[ind] = np.nan

ind = np.isnan(ds_WOD.nitrate)
NITRATE_WOD[ind] = np.nan

ind = np.isnan(ds_AWG.phosphate)
PHOSPHATE_WOD[ind] = np.nan

100%|██████████| 684/684 [02:24<00:00,  4.87it/s]


# SAVE

In [13]:
ds_out = xr.Dataset(coords={'month': month_WOD,
                        'pres': ds_WOA.pres})

# add to dataset
ds_out['temp'] = xr.DataArray(TEMP_WOD,dims = ['month','pres'],coords =[month_WOD,ds_WOA.pres])
ds_out['sal'] = xr.DataArray(PSAL_WOD,dims = ['month','pres'],coords =[month_WOD,ds_WOA.pres])
ds_out['doxy'] = xr.DataArray(DOXY_WOD,dims = ['month','pres'],coords =[month_WOD,ds_WOA.pres])
ds_out['nitrate'] = xr.DataArray(NITRATE_WOD,dims = ['month','pres'],coords =[month_WOD,ds_WOA.pres])
ds_out['phosphate'] = xr.DataArray(PHOSPHATE_WOD,dims = ['month','pres'],coords =[month_WOD,ds_WOA.pres])
ds_out['lon'] = xr.DataArray(lon_WOD,dims = ['month'],coords =[month_WOD])
ds_out['lat'] = xr.DataArray(lat_WOD,dims = ['month'],coords =[month_WOD])
ds_out 


# delete if already present
if os.path.isfile(outfn):
    os.remove(outfn)

ds_out.to_netcdf(outfn,mode='w',format = "NETCDF4")
